In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

# 
import numpy as np
import matplotlib.pyplot as plt
import os

from utils.utils import *
from tqdm import tqdm, trange


Autosaving every 180 seconds


In [2]:
######################################################################
######### GENERATE REWARDED (04) AND UNREWARDED (02) TIMES ###########
######################################################################

# this function is for making rewarded/unrewarded times and generating the final data+label .npz files
root_dir = '/media/cat/1TB/yuki/'
animal_id = 'IA1'

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ['IA1']
for animal_id in animal_ids:
    sessions = os.listdir(os.path.join(root_dir,
                                      animal_id,
                                      'tif_files'))

    # 
    sessions = [
       # 'IA1pm_Feb9_30Hz',
        'IA1pm_Feb10_30Hz',
        'IA1pm_Feb11_30Hz'        
    ]
    
    # 
    for session in tqdm(sessions):
        print ("session: ", session)

        # 
        res = get_04_02_triggers(os.path.join(root_dir,animal_id), session)

        if res==False:
            print ("no res")
            continue

        ##########################################
        ######## LOAD DATA FOR LEVER PULLS #######
        ##########################################
        if True:
        #try: 
            (data_segments, 
              data_segments_random,
              data_continuous, 
              rewarded_pull_times, 
              nonrewarded_pull_times, 
              names) = load_rois_lever_pulls(root_dir, animal_id, session)
        else:
        #except:
            print ("Whole stack missing, ...")
            continue

        ###############################################
        ######## LOAD DATA FOR BODY MOVEMENTS #########
        ###############################################
        if True:
        #try: 
             (data_segments, 
              data_segments_random,
              data_continuous, 
              movement_times, 
              names) = load_rois_body_movements(root_dir, animal_id, session)
        else:
        #except:
            print ("Whole stack missing, ...")
            continue

            
        ###############################################
        ######## VISUALIZE RESULTS - OPTIONAL #########
        ###############################################
        # 
        if False:
            visualize_loaded_data(data_continuous, 
                                  data_segments,
                                  rewarded_pull_times,
                                  nonrewarded_pull_times,
                                  names)
            plt.show()

            input("Press Enter to continue...")


        ######################################
        ######### MAKE TRAINING DATA #########
        ######################################

        # 
        labels = np.zeros((data_continuous.shape[0], 2), 'int32').T

        labels[0,rewarded_pull_times]=1
        labels[1,nonrewarded_pull_times]=1

        #

        # save data
        training_names = ['rewarded_lever_pulls','nonrewarded_lever_pulls']

        # 
        fname_out = os.path.join(root_dir,
                                 animal_id,
                                 'tif_files',
                                 session,
                                 session+"_rnn_training.npz")

        # 
        print ("fnameout: ", fname_out)
        np.savez(fname_out,
                data_continuous = data_continuous,
                data_segments = data_segments,
                data_segments_random = data_segments_random, 
                data_names = names,
                training_labels = labels,
                training_names = training_names)


        
        

100%|██████████| 2076/2076 [00:00<00:00, 19702.62it/s]


session:  IA1pm_Feb10_30Hz
[5.136 4.728 4.736]
(7, 19863)
INITS:  [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0 (2076,)
[   98   105   188 ... 19850 19851 19860]
[  196   210   376 ... 39700 39702 39720]
1 (2043,)
[    7    17    50 ... 19852 19853 19857]
[   14    34   100 ... 39704 39706 39714]
2 (3834,)
[    2     3     5 ... 19858 19859 19860]
[    4     6    10 ... 39716 39718 39720]
3 (3532,)
[    3     4     7 ... 19850 19851 19859]
[    6     8    14 ... 39700 39702 39718]
4 (1363,)
[   74   233   244 ... 19846 19847 19848]
[  148   466   488 ... 39692 39694 39696]
5 (777,)
[  192   194   197   203   205   223   225   226   228   231   242   243
   244   245   246   248   251   253   254   257   261   263   264   266
   267   268   269   270   275   277   278   279   291   312   313   314
   333   334   335   336   347   389   392   415   498   506   860   862
   864 


100%|██████████| 2043/2043 [00:00<00:00, 11474.06it/s]

area:  0   (1999, 1800, 16) (1999, 1800, 16)




  3%|▎         | 130/3834 [00:00<00:02, 1292.71it/s]

area:  1   (1954, 1800, 16) (1954, 1800, 16)



  0%|          | 0/2 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb9_30Hz
/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz_locanmf_wholestack.npz


########################################
######### LOAD DATA AND LABELS #########
########################################

fname = '/home/cat/IA1am_Mar4_30Hz_rnn_training.npz'

data = np.load(fname)
data_continuous = data['data_continuous']
data_segments = data['data_segments']
data_names = data['data_names']
training_labels = data['training_labels']
training_names = data['training_names']

print ("Data continuous: ", data_continuous.shape)
print ("data names: ", data_names)
print ('')
print ("training labels: ", training_labels.shape)
print ("training names: ", training_names)


In [120]:
##############################################
##############################################
##############################################

data = np.load('/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz_3secNoMove_movements.npz',
              allow_pickle=True)

labels = data['labels']
print (labels)

shift = data['video_shift']
print (shift)

feature_quiescent = data['feature_quiescent']
print (feature_quiescent.shape)

inits = data['feature_initiations']
print ("inits: ", inits.shape)

allq = data['all_quiescent']
print ("All quiescent: ", allq.shape)

fig = plt.figure()
plt.imshow(inits, aspect='auto')
plt.show()



['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
[5.136 4.728 4.736]
(7,)
inits:  (7, 19863)
All quiescent:  (25, 2)


In [12]:
###############################################
######## LOAD DATA FOR BODY MOVEMENTS #########
###############################################
root_dir = '/media/cat/1TB/yuki/'
animal_id = 'IA1'
session = 'IA1pm_Feb10_30Hz'

# 
(data_segments, 
data_segments_random,
data_continuous, 
movement_times, 
names) = load_rois_body_movements(root_dir, 
                                  animal_id, 
                                  session)

print (names)
            
print ("Data continuous: ", data_continuous.shape)

100%|██████████| 2076/2076 [00:00<00:00, 19738.53it/s]

[5.136 4.728 4.736]
(7, 19863)
INITS:  [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0 (2076,)
[   98   105   188 ... 19850 19851 19860]
[  196   210   376 ... 39700 39702 39720]
1 (2043,)
[    7    17    50 ... 19852 19853 19857]
[   14    34   100 ... 39704 39706 39714]
2 (3834,)
[    2     3     5 ... 19858 19859 19860]
[    4     6    10 ... 39716 39718 39720]
3 (3532,)
[    3     4     7 ... 19850 19851 19859]
[    6     8    14 ... 39700 39702 39718]
4 (1363,)
[   74   233   244 ... 19846 19847 19848]
[  148   466   488 ... 39692 39694 39696]
5 (777,)
[  192   194   197   203   205   223   225   226   228   231   242   243
   244   245   246   248   251   253   254   257   261   263   264   266
   267   268   269   270   275   277   278   279   291   312   313   314
   333   334   335   336   347   389   392   415   498   506   860   862
   864   946  1022  1023  1095  11


100%|██████████| 2043/2043 [00:00<00:00, 12797.40it/s]

area:  0   (1999, 1800, 16) (1999, 1800, 16)



  3%|▎         | 129/3834 [00:00<00:03, 1124.75it/s]

...

area:  2   (3661, 1800, 16) (3661, 1800, 16)


100%|██████████| 1363/1363 [00:00<00:00, 8989.17it/s]

area:  3   (3431, 1800, 16) (3431, 1800, 16)



100%|██████████| 66/66 [00:00<00:00, 51055.71it/s]
/home/cat/code/widefield/yueqi/utils/utils.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  stack_random = np.array(stack_random)
/home/cat/code/widefield/yueqi/utils/utils.py:273: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print ("stacK: ", stack.shape, stack[0].shape, ' random: ', stack_random.shape)


area:  4   (1343, 1800, 16) (1343, 1800, 16)
area:  5   (733, 1800, 16) (733, 1800, 16)
area:  6   (64, 1800, 16) (64, 1800, 16)
stacK:  (7,) (1999, 1800, 16)  random:  (7,)
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
Data continuous:  (40000, 16)


In [4]:
print (len (data_segments[1]))
print (len(movement_times),len(movement_times[0]))


#
body_id = 0

print (movement_times[body_id])

lockout = 3*30

idx2 = []
idx2.append(0)
for k in range(1,len(movement_times[body_id]),1):
    if (movement_times[body_id][k]-movement_times[body_id][k-1])>lockout:
        idx2.append(k)
        
print (idx2)
print (movement_times[body_id][idx2])


2024
7 2072
[  376   380   382 ... 39696 39698 39700]
[0, 66, 68, 72, 78, 92, 110, 150, 158, 238, 250, 258, 270, 278, 289, 482, 528, 637, 647, 649, 655, 659, 673, 681, 731, 758, 777, 787, 797, 805, 830, 834, 835, 836, 838, 839, 876, 894, 895, 896, 948, 992, 1010, 1017, 1121, 1173, 1178, 1189, 1205, 1207, 1217, 1228, 1255, 1303, 1325, 1535, 1572, 1584, 1596, 1605, 1607, 1608, 1611, 1621, 1668, 1681, 1688, 1693, 1700, 1708, 1709, 1710, 1719, 1723, 1727, 1736, 1741, 1742, 1743, 1746, 1748, 1752, 1754, 1756, 1759, 1761, 1762, 1776, 1778, 1782, 1783, 1798, 1812, 1820, 1844, 1846, 1848, 1872, 1874, 1878, 1881, 1948, 1951, 1952, 1955, 1958, 1977, 1979, 1997, 2036, 2050, 2063, 2065, 2066, 2067]
[  376  1410  1666  1882  2176  2388  2636  2964  3206  3798  3982  4252
  4626  5060  5510  6256  6694  7326  7688  7954  8092  8260  8696  8960
  9256  9556  9798 10310 10630 11092 11462 12356 12630 12738 13008 13198
 13622 13962 14064 14368 14722 15040 15248 15794 16456 16822 16956 17234
 17524 17674

In [11]:
# 
print (movement_times[body_id].shape)
d = np.array(data_segments[body_id])
print (d.shape)


#d = np.roll(d, int(5000),axis=1)

roi_id = 1
#plt.plot(d[idx2].mean(0)[:,roi_id],c='blue',label='body mov')

# plt.plot(d[idx2,:,roi_id].T,
#          c='blue',
#          label='body mov',
#         alpha=.1)

mean = d[idx2,:,roi_id].mean(0)
plt.plot(mean,
         c='black',
         linewidth=3,
         alpha=1)

#
d = np.array(data_segments_random[body_id])
mean = d[idx2,:,roi_id].mean(0)
plt.plot(mean,
         c='red',
         linewidth=3,
         alpha=1)

print (d.shape)

# plt.plot(d.mean(0)[:,roi_id], c='red',label='random')
# plt.legend()
# plt.plot([0,1800],[0,0],'--',c='grey')
# plt.plot([900,900],[-100,100],'--',c='grey')
# plt.xlim(0,1800)
# plt.ylim(-50,50)
plt.show()

(2072,)
(2072, 600, 16)
(2072, 600, 16)


In [5]:
s1 = 'Feb10_'; s2='IA1pm_Feb10_30Hz'

if s1 in s2:
    print (True)

In [117]:
print (data_continuous.shape)
print (movement_times[0])

(40000, 16)
[  941   942   943 ... 19850 19851 19860]
